In [5]:
%%bigquery
-- 1) Quarterly mapping: ZIP → County FIPS/LAUS → LMA by YEAR & QUARTER
CREATE OR REPLACE TABLE `sevis-beta.supporting_data.zip_county_lma_quarterly` AS
WITH lma AS (
  SELECT
    LPAD(CAST(SAFE_CAST(TRIM(CAST(County_LAUS_areacode AS STRING)) AS INT64) AS STRING), 5, '0')
      AS county_laus_code5,
    Labor_Market_Area_name                      AS lma_name,
    County_Name_State_Abbreviation              AS county_name_state_abbrev,
    LPAD(CAST(SAFE_CAST(TRIM(CAST(State_FIPS_Code  AS STRING)) AS INT64) AS STRING), 2, '0') AS state_fips2,
    LPAD(CAST(SAFE_CAST(TRIM(CAST(County_FIPS_Code AS STRING)) AS INT64) AS STRING), 3, '0') AS county_fips3,
    state_name
  FROM `sevis-beta.supporting_data.working_pop_by_county_fips_2004-2023`
),
zipcty AS (
  SELECT
    LPAD(CAST(SAFE_CAST(TRIM(CAST(ZIP    AS STRING)) AS INT64) AS STRING), 5, '0') AS zip5,
    LPAD(CAST(SAFE_CAST(TRIM(CAST(COUNTY AS STRING)) AS INT64) AS STRING), 5, '0') AS county_fips5,
    YEAR,
    QUARTER,
    SOURCE_FILE
  FROM `sevis-beta.supporting_data.HUD_zip_code_to_county_crosswalk_2010-2024`
)
SELECT DISTINCT
  z.YEAR,
  z.QUARTER,
  z.zip5,
  z.county_fips5                                        AS county_fips5,
  SUBSTR(z.county_fips5, 1, 2)                          AS state_fips2,
  SUBSTR(z.county_fips5, 3, 3)                          AS county_fips3,
  l.lma_name,
  l.county_name_state_abbrev,
  l.state_name,
  z.SOURCE_FILE
FROM zipcty z
JOIN lma l
  ON z.county_fips5 = l.county_laus_code5
;

Query is running:   0%|          |

""
